In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
import joblib
import sys

In [ ]:
#def sequence_file(tile_num):

#import Megan's data
Data_path = "/u/home/m/mardren/scratch/SequenceML"
gkm_path = "%s/features.gkSVM.HepG2.tsv.gz"%Data_path
conv_path       = "%s/features.dragoNN_ConvModel.HepG2.SV40P.Rep1.tsv.gz"%Data_path
deepfact_path       = "%s/features.dragoNN_DeepFactorizedModel.HepG2.SV40P.Rep1.tsv.gz"%Data_path

#read megan's data into dataframes
gkm = pd.read_csv(gkm_path, header = 0, index_col = 0, sep = '\t')
conv  = pd.read_csv(conv_path, header = 0, index_col = 0, sep = '\t')
deepfact = pd.read_csv(deepfact_path, header = 0, index_col = 0, sep = '\t')
    

#read in Mudi's data
tile_scores = pd.read_pickle('/u/home/m/mudiyang/scratch/Tile_Prediction_Regressors/Data/all_data.pkl')


#reindex everything to have the same index and columns
index = tile_scores.index
columns = tile_scores.columns

gkm = gkm.reindex(index)
conv = conv.reindex(index)
deepfact = deepfact.reindex(index)

gkm.columns = columns
conv.columns = columns
deepfact.columns = columns

In [ ]:
#create mega dataframe of all features
data = pd.concat([conv,gkm,deepfact, tile_scores], axis=1)

#split into train and test data: chr8 and 18 are used as test
chr_8 = np.genfromtxt('/u/home/m/mudiyang/scratch/Scaleup_counts_sequences/DNACOUNTS/chr8.txt', 
                        dtype=str, delimiter ='\n')
chr_18 = np.genfromtxt('/u/home/m/mudiyang/scratch/Scaleup_counts_sequences/DNACOUNTS/chr18.txt', 
                        dtype=str, delimiter ='\n')
drop_set = np.concatenate((chr_8, chr_18), axis=None)


test_set = data[data.index.isin(drop_set)]
train_set = data[~data.index.isin(drop_set)]

test_set = test_set.dropna()
train_set = train_set.dropna()

In [ ]:
#train a regressor for each tile, save regressor
for i in range(1,32,1):
    tile = i
    
    y_train = pd.DataFrame(train_set['t%s'%tile])
    y_train = y_train.iloc[:,3]
    y_test = pd.DataFrame(test_set['t%s'%tile])
    y_test = y_test.iloc[:,3]

    X_train = train_set.drop(columns = ['t%s'%tile])
    X_test = test_set.drop(columns = ['t%s'%tile])
    
    #convert train and test sets to arrays
    X_train = X_train.values
    X_test = X_test.values
    y_train = y_train.values.flatten()
    y_test = y_test.values.flatten()
    
    from sklearn.ensemble import RandomForestRegressor

    tile_predictor = RandomForestRegressor(bootstrap=True, max_depth=10, max_features=20, n_estimators = 99)
    tile_predictor.fit(X_train, y_train)

    input_subset = X_train[:5]
    labels_subset = y_train[:5]

    print("Predictions are: ", tile_predictor.predict(input_subset))
    print("Correct Labels:  ", list(labels_subset))

    #check mean_squared_error
    from sklearn.metrics import mean_squared_error
    test_predictions = tile_predictor.predict(X_test)
    test_mse = mean_squared_error(test_predictions, y_test)
    test_rmse = np.sqrt(test_mse)

    train_predictions = tile_predictor.predict(X_train)
    train_mse = mean_squared_error(train_predictions, y_train)
    train_rmse = np.sqrt(train_mse)

    print("Root Mean Squared error on training set is: ", train_rmse)
    print("Root Mean Squared error on test set is:     ", test_rmse)

    #flatten the arrays used for plotting
    test_predictions = test_predictions.flatten()
    y_test = y_test.flatten()


    RMS_train = "Root Mean Squared error on training set is: %s"%(train_rmse)
    RMS_test = "Root Mean Squared error on test set is: %s"%(test_rmse)

    #visualize the results
    caption = "correlation pearson r= %s"%(pd.Series(test_predictions).corr(pd.Series(y_test)))

    plt.scatter(
        x=test_predictions,
        y=y_test,
        edgecolors='w'
    )

    plt.title("Linear Inference Tile Predictor, Hep, SV40")
    plt.xlabel("test_predictions \n\n %s \n\n %s \n %s"%(caption, RMS_train, RMS_test))
    plt.ylabel("test_labels")

    plt.show()

    
    from sklearn.externals import joblib
    path = '/u/home/m/mudiyang/scratch/Tile_Prediction_Regressors/Combined/RF_all_inference/RF_all_inference_t%s.pkl'%tile
    joblib.dump(tile_predictor, path)